<a href="https://colab.research.google.com/github/adautofbn/ri_labs/blob/master/index_consulta.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [282]:
!pip install memory_profiler

import pandas as pd
import nltk
import re
import collections
import heapq
import time
from nltk.tokenize import RegexpTokenizer

%load_ext memory_profiler

nltk.download('stopwords')
result = pd.read_csv('https://raw.githubusercontent.com/adautofbn/ri_lab_01/master/output/results.csv') # Resultados adquiridos do site brasil-247

The memory_profiler extension is already loaded. To reload it, use:
  %reload_ext memory_profiler
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


### 1. Execute o algoritmo ilustrado na Fig. 5.8 do livro texto (pag. 157) para gerar um índice similar o mostrado na Fig. 5.4 (pag. 134). Guarde o índice em disco em formato csv.

In [0]:
tknz = RegexpTokenizer(r'([A-Za-zÁáÉéÍíÓóÚúÃãÕõÇçÂâÊê]{3,27})')
stopwords = nltk.corpus.stopwords.words('portuguese') 
indexes = {}
n = 0

for text in result.text:
  words = [word for word in tknz.tokenize(text.lower())
           if not bool(re.search(r'\d', word))
           and word not in stopwords and len(word) >= 3]  
  n += 1
  for t in words:
    if t not in indexes.keys():
      indexes[t] = []
    indexes[t].append(n)
    
for elem in indexes.items():
  d = dict(collections.Counter(elem[1]))
  indexes[elem[0]] = list(d.items())

Tokenização feita a partir de regex (aceitando palavras em português) e filtragem de palavras sem significado expressivo contidas na lista de stopwords.

Utilizando dicionário foi possível realizar a indexação dos tokens adquiridos, a cada novo documento tokenizado:
 * Númeração do documento a ser processado;
 * Remoção de palavras repetidas no documento fazendo o cast do array de tokens para um set;
 * As palavras que aparecem pela primeira são adicionadas como chave;
 * As palavras que já estão no dicionário tem o número do documento acrescentado nos valores da sua chave.

In [0]:
tokens_table = pd.DataFrame()

tokens_table['Word'] = [word for word in indexes.keys()]
tokens_table['Documents'] = [docs for docs in indexes.values()]

tokens_table.to_csv('ranking.csv', encoding='utf-8', index=False)

### 2. Implemente as abordagens de processamento de consulta documento-por-vez e termo-por-vez (Fig. 5.16 e 5.18).

In [0]:
def doc_a_time(query, indexes, k):
    start = time.time()
    query_indexes = []

    rank = []
    for word in query.split(" "):
      if word in indexes.keys():
          query_indexes.append(indexes[word])
    for doc in range(1, len(result.text)+1):
      doc_score = 0
      for index in query_indexes:
          for i in index:
              if i[0] == doc:
                  doc_score += i[1]
                  break
      if doc_score != 0:
        heapq.heappush(rank, (doc_score, doc))
    
    end = time.time() - start
    return heapq.nlargest(k, rank), end

In [0]:
def term_a_time(query, indexes, k):
    start = time.time()
    
    a = {}
    query_indexes = []
    rank = []
    for word in query.split(" "):
      if word in indexes.keys():
          query_indexes.append(indexes[word])
    for index in query_indexes:
      for i in index:
          doc = i[0]
          if doc not in a.keys():
              a[doc] = 0
          a[doc] += i[1]
    for (doc, score) in a.items():
      heapq.heappush(rank, (score, doc))
        
    end = time.time() - start
    return heapq.nlargest(k, rank), end

In [0]:
queries = ["bolsonaro", "educação", "ministério", "brasil", "economia"]
k = 10
results_doc = []
results_term = []
times_doc = []
times_term = []

for query in queries:
  score_doc, time_doc = doc_a_time(query, indexes, k)
  results_doc.append(score_doc)
  times_doc.append(time_doc)
  
  score_term, time_term = term_a_time(query, indexes, k)
  results_term.append(score_term)
  times_term.append(time_term)

In [288]:
queries_df = pd.DataFrame()
queries_df['Consulta'] = queries
queries_df['Documento por vez'] = results_doc
queries_df['Termo por vez'] = results_term
queries_df['Comparativo'] = queries_df['Documento por vez'] == queries_df['Documento por vez']
queries_df.index+=1
queries_df

,Consulta,Documento por vez,Termo por vez,Comparativo
1,bolsonaro,"[(10, 62), (6, 49), (5, 70), (5, 63), (5, 28),...","[(10, 62), (6, 49), (5, 70), (5, 63), (5, 28),...",True
2,educação,"[(2, 56), (2, 12), (1, 58), (1, 32), (1, 15)]","[(2, 56), (2, 12), (1, 58), (1, 32), (1, 15)]",True
3,ministério,"[(6, 60), (3, 12), (2, 61), (2, 31), (2, 30), ...","[(6, 60), (3, 12), (2, 61), (2, 31), (2, 30), ...",True
4,brasil,"[(11, 59), (7, 66), (7, 33), (7, 21), (7, 7), ...","[(11, 59), (7, 66), (7, 33), (7, 21), (7, 7), ...",True
5,economia,"[(6, 6), (3, 32), (2, 29), (2, 8), (2, 3), (1,...","[(6, 6), (3, 32), (2, 29), (2, 8), (2, 3), (1,...",True


In [289]:
queries_tempo_df = pd.DataFrame()
queries_tempo_df['Tempo Médio: Documento por vez (ms)'] = times_doc
queries_tempo_df['Tempo Médio: Termo por vez (ms)'] = times_term
queries_tempo_df.index+=1
queries_tempo_df

,Tempo Médio: Documento por vez (ms),Tempo Médio: Termo por vez (ms)
1,0.000242,0.000028
2,0.000057,0.000005
3,0.000103,0.000013
4,0.000157,0.000023
5,0.000110,0.000014


In [290]:
print('Documento por vez uso de memória:')
%memit doc_a_time(queries[0], indexes, k)

print('Termo por vez uso de memória:')
%memit term_a_time(queries[0], indexes, k)

Documento por vez uso de memória:
peak memory: 198.37 MiB, increment: 0.00 MiB
Termo por vez uso de memória:
peak memory: 198.37 MiB, increment: 0.00 MiB


### 3. Implemente uma das versões de consulta conjuntiva (AND) (Fig. 5.20 ou 5.21).

In [0]:
def conj_query(query, indexes, k):
  
  query_indexes = []
  rank = []
  
  for word in query.split(" "):
    if word in indexes.keys():
        query_indexes.append(indexes[word])
  
  all_query = [item for sublist in query_indexes for item in sublist]
  all_query.sort()
  
  for i in range(len(all_query)):
    doc_score = 0
    d = all_query.pop()
    repeat = 1
    for index in all_query:
      if index[0] == d[0]:
        doc_score += index[1]
        repeat += 1
    if doc_score != 0 and repeat == len(query_indexes):
      doc_score += d[1]
      heapq.heappush(rank, (doc_score, d[0]))
        
  return heapq.nlargest(k, rank)

In [0]:
queries_conj = ["decisão sobre", "ministro paulo", "brasil país", "jair bolsonaro", "presidente disse"]
k = 10
results_conj = []

for query in queries_conj:
  score_conj = conj_query(query, indexes, k)
  results_conj.append(score_conj)

In [293]:
conj_queries_df = pd.DataFrame()
conj_queries_df['Consulta'] = queries_conj
conj_queries_df['Resultado'] = results_conj
conj_queries_df.index+=1
conj_queries_df

,Consulta,Resultado
1,decisão sobre,"[(6, 65), (6, 55), (5, 8), (5, 5), (4, 62), (4..."
2,ministro paulo,"[(6, 56), (6, 8), (5, 75), (5, 68), (5, 14), (..."
3,brasil país,"[(14, 59), (11, 33), (9, 21), (9, 7), (8, 66),..."
4,jair bolsonaro,"[(12, 62), (8, 49), (7, 70), (6, 66), (6, 63),..."
5,presidente disse,"[(9, 55), (9, 12), (8, 63), (8, 59), (7, 70), ..."


#####Dê evidências de que sua implementação está correta.

  No caso abaixo, podemos olhar separadamente para a lista invertida de cada termo e ver manualmente que todos os termos aparecem apenas no documento 1. O algoritmo realiza a soma dos scores de cada termo e atribui como score do documento no resultado final.

In [334]:
query = "petrobrás quer alterar estatuto"
query_terms = query.split()
terms_indexes = []

a = ['', '', '', '']
b = ['', '', '', '']

for term in query_terms:
  terms_indexes.append(indexes[term])
  
score_conj = conj_query(query,indexes,10)
score_conj = (score_conj[0][::-1])

query_terms.append('')
terms_indexes.append('')
a.append(query)
b.append(score_conj)

query_df = pd.DataFrame()
query_df['Termos'] = query_terms
query_df['Lista Indexada do termo'] = terms_indexes
query_df['Consulta'] = a
query_df['Resultado'] = b
query_df


,Termos,Lista Indexada do termo,Consulta,Resultado
0,petrobrás,"[(1, 4), (48, 1), (61, 3)]",,
1,quer,"[(1, 1), (18, 2), (37, 3), (45, 2), (59, 1)]",,
2,alterar,"[(1, 1), (63, 1)]",,
3,estatuto,"[(1, 3)]",,
4,,,petrobrás quer alterar estatuto,"(1, 9)"
